# Базовые операции с Apache Impala

## Подготовка

### Создание базы данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
DROP DATABASE IF EXISTS udf_demo
CASCADE;
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
CREATE DATABASE udf_demo;
"

### Создание таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d udf_demo -q "
CREATE TABLE udaf_table (
    id INT,
    VALUE INT
)
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d udf_demo -q "
INSERT INTO udaf_table
WITH records AS (
    select 1 id, 0 value union
    select 1, -1 union
    select 3, 6 union
    select 4, 100 union
    select 4, 10 union
    select 4, 1000
)
SELECT *
  FROM records
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d udf_demo -q "
SELECT *
  FROM udaf_table
"

## Создание UDF функции

### Подготовка

Исходный код:

In [ ]:
HOST=dind execute \
cat /root/impala-exercises/src/src/main/java/com/github/neshkeev/impala/udf/HelloUDF.java

### Сборка проекта

In [ ]:
HOST=maven execute \
mvn clean package \
    -f /root/impala-exercises/src/pom.xml \
    -l /tmp/build.log

HOST=maven execute \
tail /tmp/build.log

### Деплой бинарного файла

Расположение jar файла:

In [ ]:
HOST=dind execute \
ls /root/impala-exercises/src/target/*.jar

Необходимо скопировать jar файл с кодом UDF функции в Impala:

In [ ]:
UDF_JAR_PATH=$(HOST=dind execute ls /root/impala-exercises/src/target/*.jar)

docker compose cp maven:${UDF_JAR_PATH} /tmp/impala-udf.jar &&
docker compose cp /tmp/impala-udf.jar impala:/tmp/impala-udf.jar

### Применение UDF функции

Сначала необходимо создать новую функцию в Impala:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d udf_demo -q "
CREATE FUNCTION hello(string)
    RETURNS STRING
    LOCATION '/tmp/impala-udf.jar'
    SYMBOL='com.github.neshkeev.impala.udf.HelloUDF';
"

Теперь можно использовать функцию:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d udf_demo -q "
SELECT hello('world') msg;
"

### Вывод

1. Apache Impala позволяет создать функции на основе UDF функций для Apache Hive;
1. Для создания функции сначала необходимо:
    1. написать Java класс, который расшияет `org.apache.hadoop.hive.ql.exec.UDF`;
    1. упаковать Java класс в jar файл;
    1. скопировать jar файл на компьютер, где запущена база данных Impala;
    1. создать новую функцию через `CREATE FUNCTION`.

### Задание

1. Создать функцию `redacted`, которая заменяет текст на 5 символов звездочки (`*`). Пример:
```sql
SELECT redacted('Hello, World!') -- печает: *****
```
2. Создать функцию `redacted_with`, которая в дополние к логике `redacted`, позволяет указать символ для сокрытия информации (в `redacted` использовался символ `*`). Пример:
```sql
SELECT redacted_with('Hello, World!', '-') -- печатает -----
```
3. Добавить два дополнительных параметра `start`, `size`, которые заменяют `size` символов текста указанным символом, начиная с индекса `start`. Пример:
```sql
SELECT redacted_with('Hello, World!', '_', 3, 4) -- печатает He____ World!
```

## UDAF - Пользовательские функции агрегации

Impala не поддерживает пользовательские UDAF, но можно использовать Hive.

Реализация полностью новой UDAF функции является нетривиально задачей, поэтому для примера можно воспользоваться уже готовой реализацией функции `max` из стандартной поставки Apache Hive: [GenericUDAFMax](https://github.com/apache/hive/blob/9b4ea7affa4902fc2849f1a88b68103940fc9866/ql/src/java/org/apache/hadoop/hive/ql/udf/generic/GenericUDAFMax.java).

### Создание UDAF функции

Инструкция `CREATE TEMPORARY FUNCTION` означает, что функция будет доступна на время текущей сессии:

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
-- регистрация временной фукнции my_max
CREATE TEMPORARY FUNCTION my_max
    AS
    'org.apache.hadoop.hive.ql.udf.generic.GenericUDAFMax';

SELECT my_max(value) my_max, max(value) max, id
  FROM udf_demo.udaf_table
 GROUP BY id;
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql

После завершения Hive сессии функция `my_max` больше недоступна:

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
SELECT my_max(value) my_max, max(value) max, id
  FROM udf_demo.udaf_table
 GROUP BY id;
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql || true

### Постоянные UDAF функции

Для сохранения функции между сеансами необходимо зарегистрировать функцию через `CREATE FUNCTION` (без `TEMPORARY`):

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
-- регистрация постоянной фукнции my_max
CREATE FUNCTION my_max
    AS
    'org.apache.hadoop.hive.ql.udf.generic.GenericUDAFMax'
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql

Проверка доступности функции `my_max` в новых сеансах:

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
SELECT my_max(value) my_max, max(value) max, id
  FROM udf_demo.udaf_table
 GROUP BY id;
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
SELECT my_max(value) OVER(PARTITION BY id) my_max, max(value) OVER(PARTITION BY id) max, id
  FROM udf_demo.udaf_table
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql

### Вывод

1. Impala не позволяет регистрировать пользовательские UDAF функции;
1. Собственные UDAF функции можно регистрировать в Apache Hive и использовать их из Apache Hive для выполнения запросов к таблицам Apache Impala.

### Задание

1. Зарегистрировать временную функцию `my_avg`, которая должна действовать как стандартная `avg`. За основу можно взять класс [`org.apache.hadoop.hive.ql.udf.generic.GenericUDAFAverage`](https://github.com/apache/hive/blob/9b4ea7affa4902fc2849f1a88b68103940fc9866/ql/src/java/org/apache/hadoop/hive/ql/udf/generic/GenericUDAFAverage.java)
1. Убедиться, что функция `my_avg` возвращает результаты аналогичные `avg`;
1. Зарегистрировать постоянную функцию `my_permanent_min`. За основу взять класс [`org.apache.hadoop.hive.ql.udf.generic.GenericUDAFMin`](https://github.com/apache/hive/blob/9b4ea7affa4902fc2849f1a88b68103940fc9866/ql/src/java/org/apache/hadoop/hive/ql/udf/generic/GenericUDAFMin.java);
1. Убедиться, что:
    1. функция `my_permanent_min` возвращает результаты аналогичные `min`;
    1. сохраняется между сеансами.